In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# from sklearn import model_selection, linear_model, metrics
# from xgboost import XGBRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, max_error
# from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.model_selection import cross_val_score 
# from sklearn.linear_model import LinearRegression

## Building assistance lines

In [ ]:
exp_data1=pd.DataFrame({
'SO4':[3.00,2.51,1.84,1.42,1.16,1.05],
'FA':[43.9,36,28,21,15,9.9],
'DCP':[0.0,1.61,3.26,4.94,6.66,8.42]})

exp_data2=pd.DataFrame({
'SO4':[3.04,2.23,1.61,1.15,0.93,0.84],
'FA':[50.0,41,33,25,18,12],
'DCP':[0.0,1.62,3.27,4.96,6.69,8.45]})

exp_data3=pd.DataFrame({
'SO4':[2.97,2.19,1.56,1.09,0.85,0.76],
'FA':[55,46,38,29,22,15],
'DCP':[0.0,1.62,3.29,5.00,6.71,8.49]})

In [ ]:
def fill_each_line(df, l):
    n=l+1 #l=0 ==> no addition
    col=df.columns
    new_df=pd.DataFrame(columns=col)
    for i in range(len(df)-1):
        row1=df.iloc[i]
        row2=df.iloc[i+1]
        for m in range(n):
            new_row=pd.DataFrame({
                      'SO4':[row1.SO4-m*(row1.SO4-row2.SO4)/n],
                      'FA':[row1.FA-m*(row1.FA-row2.FA)/n],
                      'DCP':[row1.DCP-m*(row1.DCP-row2.DCP)/n],
            })
            new_df=pd.concat([new_df,new_row])
    new_df=pd.concat([new_df,pd.DataFrame([df.iloc[-1]])])
    new_df.reset_index(inplace=True, drop=True)
    return new_df

In [ ]:
n=0 #number of points added
exp_filled_data1=fill_each_line(exp_data1, n)
exp_filled_data2=fill_each_line(exp_data2, n)
exp_filled_data3=fill_each_line(exp_data3, n)
# exp_filled_data1

In [ ]:
def fill_between_lines(df1,df2,n):
    set=[]
    for m in range(n):
        new_df=pd.DataFrame()
        for i in range(len(df1)):
            row1,row2=df1.iloc[i],df2.iloc[i]
            new_row=pd.DataFrame({
                      'SO4':[(row1[0]-(m+1)*(row1[0]-row2[0])/(n+1))],
                      'FA':[(row1[1]-(m+1)*(row1[1]-row2[1])/(n+1))],
                      'DCP':[(row1[2]-(m+1)*(row1[2]-row2[2])/(n+1))]})
            new_df=pd.concat([new_df,new_row])
        new_df.reset_index(inplace=True, drop=True)
        set.append(new_df)
    return set

In [ ]:
m=0 #number of lines added
set12=fill_between_lines(exp_filled_data1,exp_filled_data2,m)
set23=fill_between_lines(exp_filled_data2,exp_filled_data3,m)
# set12

In [ ]:
full_set=[exp_filled_data1]+set12+[exp_filled_data2]+set23+[exp_filled_data3] #list of pd.DataFrames
# full_set

In [ ]:
def exp_to_model(df):
    col=['SO4','FA','DCP','y_SO4','y_FA']
    new_df=pd.DataFrame(columns=col)
#     DCP=0
    for n in range(len(df)):
        DCP=df.iloc[n].DCP
        for m in range(n,len(df)):
            row1,row2=df.iloc[n],df.iloc[m]
            new_row=pd.DataFrame({
                      col[0]:[row1[0]],
                      col[1]:[row1[1]],
                      col[2]:[row2[2]-DCP], #תוספת DCP בלבד
                      col[3]:[row2[0]], #ערכים שאחרי תוספת DCP
                      col[4]:[row2[1]]})
            new_df=pd.concat([new_df,new_row])
#         DCP=new_df.iloc[n]['DCP']
#         print (DCP)
    new_df.reset_index(inplace=True, drop=True)
    return new_df            

In [ ]:
full_set_model=[exp_to_model(df) for df in full_set] #list of pd.DataFrames
# full_set_model

In [ ]:
l=len(full_set_model[0])
set=[]
set_len=1
i=0
while i <l:
    new_set=[]
    for j in range(set_len):
        i+=1
        new_set.append(l-i)
    set.append(new_set)
    set_len+=1
# set

## Building contineous experiments

In [ ]:
exp5=[
[2.96,	56.1,	0],
[1.01,	23.0,	4.30],
[0.98,	20.9,	5.05],
[1.00,	21.2,	5.81],
[0.92,	19.1,	6.57],
]
exp5=pd.DataFrame(exp5, columns=['SO4','FA','DCP'])
exp5

In [ ]:
exp6=[
[2.83,	63.1,	0],
[1.11,	28.2,	4.30],
[0.98,	25.3,	5.05],
[0.97,	22.9,	5.81],
[0.97,	21.9,	6.57],
]
exp6=pd.DataFrame(exp6, columns=['SO4','FA','DCP'])
exp6

## Vizualization

In [ ]:
fig = go.Figure()

x, y, z = np.mgrid[0:3:40j, 6:50:5j, 0:10:40j]
values = -x*1 - 0.44*(y-30)
fig.add_trace(go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=values.flatten(),
    isomin=5,
    isomax=10,
    surface_count=6,
    opacity=0.1,
    colorscale='Brwnyl',
    colorbar_title = '%P2O5<br>solid',
    caps=dict(x_show=False, y_show=False, z_show=True),
    slices_z=dict(show=True, locations=[0]),
    ))

# building the reference lines with n,m fillings
for i in range(len(full_set_model)):
    df=full_set_model[i]
    for j in range(len(set)):
        x=df.y_SO4[set[j]]
        y=df.y_FA[set[j]]
        z=df.DCP[set[j]]
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', line_color='black', marker_size=3, opacity=0.3))

# building 3 final dots
fig.add_trace(go.Scatter3d(x=[1.05,0.84,0.76], y=[9.9,12.1,14.9], z=[8.42,8.45,8.49], marker_size=4, marker_color='brown', mode='markers'))

# building contineous experiments results
fig.add_trace(go.Scatter3d(x=exp5.SO4, y=exp5.FA, z=exp5.DCP, mode='markers+lines', line_color='green', marker_size=4, opacity=1))
fig.add_trace(go.Scatter3d(x=exp6.SO4, y=exp6.FA, z=exp6.DCP, mode='markers+lines', line_color='yellow', marker_size=4, opacity=1))
   
fig.update_layout(
    scene = dict(
        xaxis_title='SO4',
        yaxis_title='FA',
        zaxis_title='DCP 100%_in',
        xaxis = dict(nticks=8, range=[0,3.1]),
        yaxis = dict(nticks=12, range=[6,61]),
        zaxis = dict(nticks=20, range=[0,10])),
    width=1200,height=900,
    margin=dict(r=20, b=50, l=10, t=10),
    scene_camera_eye=dict(x=-1.5, y=1.5, z=0.2))

fig.update_layout(showlegend=False)
# fig.write_html("cont exp 5-6.html")
fig.show()